In [2]:
pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = 'Fact_Customer_Orders'
 
sf_df = my_session.sql("select PRODUCT_ID,ORDER_QTY,ORDER_DATE from {}".format(table_name))
df = sf_df.to_pandas()

In [7]:
df.columns

Index(['PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [8]:
data = df[['PRODUCT_ID','ORDER_QTY','ORDER_DATE']]

In [21]:
#Convert order_date to datetime format
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

KeyError: 'ORDER_DATE'

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225563 entries, 0 to 225562
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   PRODUCT_ID  225563 non-null  object        
 1   ORDER_QTY   225563 non-null  int16         
 2   ORDER_DATE  225563 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), object(1)
memory usage: 3.9+ MB
